# 1(a)-------------------------------------------------------------------------------------

dual: 
$$
        \max_{y\epsilon\mathbb{R^m},s\epsilon\mathbb{R^n}}b^Ty \\
        s.t: A^Ty+s=c \\
        s \geqslant 0
$$

equal problem :
$$
    \min_{y\epsilon\mathbb{R^m},s\epsilon\mathbb{R^n}}-b^Ty + l_{s \geqslant 0}(s) \\
    s.t: A^Ty+s=c 
$$

Augumented Lagrange Equation :
\begin{equation}
\begin{split}
    L_{\rho}(y,s;x) &= -b^Ty + l_{s\geqslant0}(s) + x^T(A^Ty+s-c) + \frac{\rho}{2}||A^Ty+s-c||_2^2 \\
              &= -b^Ty + l_{s\geqslant0}(s) + \frac{\rho}{2}[||A^Ty+s-c||_2^2 + \frac{2}{\rho} x^T(A^Ty+s-c) + \frac{1}{\rho^2}||x||_2^2 -  \frac{1}{\rho^2}||x||_2^2] \\
              &=-b^Ty + l_{s\geqslant0}(s) + \frac{1}{2\rho}(||\rho(A^Ty+s-c)+x||_2^2-||x||_2^2)
\end{split}
\end{equation}

Augumented Lagrange Equation :
\begin{equation}
\begin{split}
    L_{\rho}(y;x) &= \inf_{s} L_{\rho}(y,s;x) \\
              &= L_{\rho}(y,s_0;x)  \ where \ \frac{\partial L_{\rho}(y,s;x)}{\partial s} = 0 \ and \  s_0 = \Pi_{s \geqslant 0}(-\frac{x}{\rho}+c-A^Ty) \\
              &= -b^Ty  + \frac{1}{2\rho}(||\rho(A^Ty-c)+x + \Pi_{s \geqslant 0}-(x+\rho(A^Ty-c))||_2^2-||x||_2^2) \\
              &= -b^Ty  + \frac{1}{2\rho}( ||\Pi_{s \geqslant 0}(x+\rho(A^Ty-c))||_2^2-||x||_2^2)
\end{split}
\end{equation}

ALM: 
$$
y^{k+1} = argmin_{y}L_{\rho}(y,x^k) ——这个子问题可以用homework5里的方法求\\
x^{k+1} = \Pi_{s \geqslant 0}(x^k+\rho(A^Ty^{k+1}-c)
$$

# 1(b)
---
- alm 
> $\frac{\partial L_{\rho}(y,x^k)}{\partial y} = -b + A(\Pi_{s \geqslant 0 }(x^k + \rho(A^Ty-c))) $

In [1]:
from __future__ import division
import numpy as np
import time
np.random.seed(132)
n=100
m=20
A = np.random.randn(m,n)
xs = np.abs(np.random.randn(n,1) * np.random.binomial(1, m/n, (n,1)))
b = A.dot(xs)
y =np.random.randn(m,1)
s =np.multiply(np.random.rand(n,1),(xs==0))
c = A.T.dot(y) + s

In [2]:
import cvxpy as cy
x = cy.Variable(n)
objective = cy.Minimize(c.T * x)
constraints = [0 <= x, A*x == b]
prob = cy.Problem(objective, constraints)
print 'CVX objective:',prob.solve()

CVX objective: 4.99222695959


In [3]:
time.time()

1483699471.201

In [4]:
class Alm () :
    def __init__(self,A=A,b=b,s=s,c=c) :
        self.A = A
        self.b = b
        self.s = s
        self.c = c
    def solve(self,x_0,y_0,p = 0.4):
        x= x_0
        y = y_0
        step = 0.001
        start = time.time()
        for i in range(1000) :
            proj = x + p * (self.A.T.dot(y)-self.c)
            proj[proj<0] = 0
            py = - self.b + self.A.dot(proj)
            while(np.sum(np.square(py)) > 0.01) :
                y -= step * py
                proj = x + p * (self.A.T.dot(y)-self.c)
                proj[proj<0] = 0
                py = - self.b + self.A.dot(proj)
            x = x + p * (self.A.T.dot(y) - self.c)
            x[x<0] = 0
        print 'Alm：objective value :',self.c.T.dot(x)
        print 'Alm x sum abs erro:',np.sum(np.abs(x-xs))
        print 'Alm used time:',time.time() - start

In [5]:
y_0 = np.random.randn(m,1)
x_0 = np.random.randn(n,1)
s_0 = np.random.randn(n,1)
model = Alm()
model.solve(x_0,y_0)
print 'CVX objective:',prob.solve()

Alm：objective value : [[ 5.02799324]]
Alm x sum abs erro: 7.07262923501
Alm used time: 0.324000120163
CVX objective: 4.99222695959


# 2(a)----------------------------------------------------------------------------------------------------

## ADMM:dual problem 
$$
min -b^Ty + l_{s \geqslant 0}(s)\\
s.t  \ \ A^Ty+s=c \\
$$

ADMM algorithom:
$$
    y^{k+1} = argmin_{y}-b^Ty + <x^k,A^Ty>+\frac{\rho}{2}||A^Ty+s^k-c||_2^2
    \\ y^{k+1} = (\rho AA^T)^{-1}(b-Ax^k-\rho A(s^k-c))
    \\ s^{k+1} = argmin_{s} l_{s\geqslant 0 }(s) + <x^k,s> +\frac{\rho}{2}||A^Ty^{k+1}+s-c||_2^2
    \\ s^{k+1} = \Pi_{s\geqslant 0}(-\frac{x^k}{\rho}+c-A^Ty^{k+1})
    \\ x^{k+1} =  x^k + \rho (A^Ty^{k+1}+s^{k+1}-c)
$$

In [6]:
class Admm (Alm) :
    def solve(self,y_0,s_0,p = 0.4) :
        x = np.zeros_like(xs)
        y = y_0
        s = s_0

        cc = 1/p * np.linalg.inv(self.A.dot(self.A.T))
        start = time.time()
        for i in range(10000) :
            y = np.dot(cc , self.b - self.A.dot(x)-p*self.A.dot(s) + p*self.A.dot(self.c)) 
            s = -x/p + c -self.A.T.dot(y)
            s[s<0.0] = 0
            x = x + p * (self.A.T.dot(y) + s -c)
        print 'ADMM：objective value :',self.b.T.dot(y)
        print 'ADMM x sum abs erro:',np.sum(np.abs(x-xs))
        print 'ADMM used time:',time.time() - start

In [7]:
model = Admm()
model.solve(y_0,s_0)
print 'CVX objective:',prob.solve()

ADMM：objective value : [[ 4.99222706]]
ADMM x sum abs erro: 3.55024258311
ADMM used time: 0.40299987793
CVX objective: 4.99222695959


## DRS:primal problem------------------------------------------------------------------------------------------------
$$
    min  \ \ c^Tx \\
    s.t \ \ Ax = b \\
    x \geqslant 0
$$

转换一下形式
$$
    min \ \ g(x) + f(x)
    \\g(x) = c^Tx + l_{Ax=b}(x)  \\ f(x) = l_{x \geqslant 0}(x)
$$

DRS algorithom :
$$
    u^+ = prox_{\rho g }(x+w) ------u^+ = (-c \rho + x + w)  + A^T(AA^T)^{-1}(b-A(-c\rho +x + w)) \\
    x^+ = prox_{\rho f}(u^+ - w) ---------------------x^+ = \Pi_{s \geqslant 0}(u^+ - w) \\
    w^+ = w + x^+ - u^+ ------------------------------
$$

In [8]:
class Drs(Alm) :
    def solve(self,x_0,p=0.3) :
        x = x_0
        w = x_0
        cc = self.A.T.dot(np.linalg.inv(self.A.dot(self.A.T)))
        start = time.time()
        for i in range(10000) :
            u = (-self.c*p+x+w)+np.dot(cc,self.b-self.A.dot(-self.c*p+x+w))
            x =  u - w
            x[x<0.0] = 0
            w = w + x - u
        print 'DRS：objective value :',self.c.T.dot(x)
        print 'DRS x sum abs erro:',np.sum(np.abs(x-xs))  
        print 'DRS used time:',time.time() - start

In [9]:
model = Drs()
model.solve(s_0)
print 'CVX objective:',prob.solve()

DRS：objective value : [[ 4.99222786]]
DRS x sum abs erro: 2.57763480283
DRS used time: 0.282999992371
CVX objective: 4.99222695959


# 2(b)---------对某个问题用ADMM等价于对它的对偶问题用DRS----------------

这里ADMM中出现的变量有$y^{k+1},s^{k+1},x^{k+1}$,DRS中出现的变量有$u^+,x^+,w^+$.它们之间的关系是：

$$
    \\
    u^+ = x^k + \rho s^{k} + \rho A^T(y^{k+1}-c) \\
    x^+ = x^{k+1} \\
    w^+ = \rho s^{k+1}
$$

### 可以结合2（a）验证关系成立。下面给出一般性的证明。

ADMM：
$$min \ \ f_1(x_1) + f_2(x_2) \\
s.t:A_1x_1 + A_2x_2 = b
$$
algorithom:
$$
    x_1^{k+1} = argmin_{x_1}f_1(x_1) + <A_1x_1,z^k> + \frac{\rho}{2}||A_1x_1 + A_2x_2^k-b||_2^2 \\
    x_2^{k+1} = argmin_{x_2}f_2(x_2) + <A_2x_2,z^k> + \frac{\rho}{2}||A_1x_1^k + A_2x_2 - b||_2^2 \\
    z^{k+1} = z^k + \rho(A_1x_1^{k+1} + A_2x_2^{k+1})
$$

DRS:
$$
    min \ \ b^Tz + f_1^*(-A_1^Tz) + f_2^*(-A_2^Tz) \\
    g(z) := b^Tz + f_1^*(-A_1^Tz)  \\
    f(z) := f_2^*(-A_2^Tz) 
$$
algorithom:
$$
    u^+ = prox_{\rho g }(z+w) \\
    z^+ = prox_{\rho f}(u^+ - w)  \\
    w^+ = w + z^+ - u^+
$$


### 对对偶问题用proximal method 等价于对原问题用ALM。从而ADMM和DRS的关系得证。
下面具体证明
- proximal method:
$$
    u^+ = prox_{\rho g}(z+w) \\  
    u^+ = argmin_x b^Tx + f_1^*(-A_1^Tx) + \frac{1}{2 \rho}||x-(z+w)||_2^2   \ \ \ \ \ \ --(1)\\
$$
- ALM:
$$
    u^+ = z+w+\rho(A_1\hat x_1-b)   \ \ \ \ \ \  \ \ \ \ \ -----------(2) \\
    \hat x_1 = argmin_{x_1}f_1(x_1) + <z+w,A_1x_1> + \frac{\rho}{2}||A_1x_1-b||_2^2 \ \ \ \ (3)
$$

**上面三个等式等价:这里通过验证最优性条件说明**，就是（2）的解满足（1）的最优性条件。

- 对(1)求导有：$b -A_1 \partial f_1^*(-A_1^Tx) + \frac{1}{\rho}(x-(z-w)) = 0 $
- 对(3)求导有：$\partial f_1(x_1) + A_1^T(z+w) + \rho A_1^T(A_1x_1-b) = 0$
- 容易证明这里(2)满足（1）的最优性条件

- proximal method 
\begin{equation} 
\begin{split} 
    z^+ &= prox_{\rho f}(u^+ - w) \\
      &=prox_{\rho f}(z+\rho(A_1 \hat x_1-b)) \\
      & = argmin_{x}f_2^*(-A_2^Tx) + \frac{1}{2\rho}||z+\rho(A_1 \hat x_1-b)-x||_2^2
\end{split}
\end{equation}
- ALM
$$
    z^+ = z + \rho(A_1 \hat x_1 - b) + \rho A_2 \hat x_2  \\
    \hat x_2 = argmin_{x_2}f_2(x_2) + <A_2x_2, z + \rho(A_1 \hat x_1-b)> + \frac{\rho}{2}||A_2x_2||_2^2
$$
类似上面的证明，可以知道结论成立。
- 从而 $w^+ = A_2 \hat x_2$

 ~~Moreau decomposition:$z^+ = z + \rho(A_1 \hat x_1 - b) + \rho \ prox_{\rho^{-1}f^*}(\frac{z}{\rho}+A_1\hat x_1 - b)$~~
